In [2]:
import numpy as np
import tensorflow as tf

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('../input/b7-dataset/MINOR PROJECT')

print (dataset_path)

print("Types of classes labels found: ", len(dataset_path))

In [3]:
class_labels = []

for item in dataset_path:
    all_classes = os.listdir('../input/b7-dataset/MINOR PROJECT' + '/' +item)
    for room in all_classes:
        class_labels.append((item, str('dataset_path' + '/' +item) + '/' + room))
        

In [4]:
df = pd.DataFrame(data=class_labels, columns=['Labels', 'image'])

In [5]:
print("Total number of images in the dataset: ", len(df))

label_count = df['Labels'].value_counts()
print(label_count)

In [6]:
import cv2
path = '../input/b7-dataset/MINOR PROJECT'
dataset_path = os.listdir('../input/b7-dataset/MINOR PROJECT')

im_size = 300

images = []
labels = []

for i in dataset_path:
    data_path = path+'/' + str(i)  
    filenames = [i for i in os.listdir(data_path) ]
   
    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
#         print(img.shape)
        
        images.append(img)
        labels.append(i)

In [7]:
images = np.array(images)
# print(images[0])
images = images.astype('float32') / 255.0
# print(images[0])
images.shape

In [8]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
y=df['Labels'].values


y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)


In [10]:
y=y.reshape(-1,1)

from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
Y = ct.fit_transform(y) #.toarray()


In [11]:
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB3

NUM_CLASSES = 2
IMG_SIZE = 300
size = (IMG_SIZE, IMG_SIZE)


inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))



outputs = EfficientNetB3(include_top=True, weights=None, classes=NUM_CLASSES)(inputs)

In [12]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


images, Y = shuffle(images, Y, random_state=1)

train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.25, random_state=415)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

In [13]:
model = tf.keras.Model(inputs, outputs)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"] )

model.summary()

hist = model.fit(train_x, train_y, epochs=30, verbose=2)

In [14]:
import matplotlib.pyplot as plt

plt.plot(hist.history["accuracy"])
# plt.plot(hist.history["val_accuracy"])
plt.title("model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["train", "validation"], loc="upper left")
plt.show()


In [15]:
preds = model.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [16]:
from matplotlib.pyplot import imread
from matplotlib.pyplot import imshow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.applications.imagenet_utils import preprocess_input


img_path = '../input/b7-dataset/MINOR PROJECT/Consolidation/00000242_000.png'



img = cv2.imread(img_path)
img = cv2.resize(img, (300, 300))

x = np.expand_dims(img, axis=0)
x = preprocess_input(x)

print('Input image shape:', x.shape)

my_image = imread(img_path)
imshow(my_image)

In [17]:
preds=model.predict(x)
preds = preds.reshape(1,-1)

In [18]:
preds

In [19]:
class ChannelAttention(tf.keras.layers.Layer):
      def _init_(self, filters, ratio):
        super(ChannelAttention, self)._init_()
        self.filters = filters
        self.ratio = ratio

        def build(self, input_shape):
            self.shared_layer_one = tf.keras.layers.Dense(self.filters//self.ratio,
                             activation='relu', kernel_initializer='he_normal', 
                              use_bias=True, 
                              bias_initializer='zeros')
            self.shared_layer_two = tf.keras.layers.Dense(self.filters,
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')

        def call(self, inputs):
            # AvgPool
            avg_pool = tf.keras.layers.GlobalAveragePooling2D()(inputs)
            

            avg_pool = self.shared_layer_one(avg_pool)
            avg_pool = self.shared_layer_two(avg_pool)



            attention = tf.keras.layers.Add()([avg_pool,max_pool])
            attention = tf.keras.layers.Activation('sigmoid')(attention)
            
            return tf.keras.layers.Multiply()([inputs, attention])

In [20]:
class SpatialAttention(tf.keras.layers.Layer):
      def _init_(self, kernel_size):
        super(SpatialAttention, self)._init_()
        self.kernel_size = kernel_size
        
        def build(self, input_shape):
            self.conv2d = tf.keras.layers.Conv2D(filters = 1,
                    kernel_size=self.kernel_size,
                    strides=1,
                    padding='same',
                    activation='sigmoid',
                    kernel_initializer='he_normal',
                    use_bias=False)

        def call(self, inputs):
            
            # AvgPool
            avg_pool = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x, axis=3, keepdims=True))(inputs)
            

            attention = tf.keras.layers.Concatenate(axis=3)([avg_pool, max_pool])

            attention = self.conv2d(attention)


            return tf.keras.layers.multiply([inputs, attention])

In [21]:
efnb3 = EfficientNetB3(weights=None, include_top=False, input_shape=train_x.shape[1:], classes=2)
simple_cnn_with_attention = tf.keras.models.Sequential()
simple_cnn_with_attention.add(efnb3)
simple_cnn_with_attention.add(tf.keras.layers.Conv2D(32, 3, input_shape= train_x.shape[1:], activation='elu', padding='same'))
simple_cnn_with_attention.add( tf.keras.layers.BatchNormalization())
simple_cnn_with_attention.add( tf.keras.layers.Conv2D(32, 3, padding='same', activation='elu'))
simple_cnn_with_attention.add( tf.keras.layers.BatchNormalization())
simple_cnn_with_attention.add( ChannelAttention())
simple_cnn_with_attention.add(SpatialAttention())
simple_cnn_with_attention.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
simple_cnn_with_attention.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='elu'))
simple_cnn_with_attention.add(tf.keras.layers.BatchNormalization())
simple_cnn_with_attention.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='elu'))
simple_cnn_with_attention.add(tf.keras.layers.BatchNormalization())
simple_cnn_with_attention.add(ChannelAttention())
simple_cnn_with_attention.add(SpatialAttention())
simple_cnn_with_attention.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
simple_cnn_with_attention.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='elu'),)

simple_cnn_with_attention.add(tf.keras.layers.BatchNormalization())

simple_cnn_with_attention.add( tf.keras.layers.Conv2D(128, 3, padding='same', activation='elu'))

simple_cnn_with_attention.add(ChannelAttention())

simple_cnn_with_attention.add(SpatialAttention())

simple_cnn_with_attention.add(tf.keras.layers.GlobalAveragePooling2D())

simple_cnn_with_attention.add(tf.keras.layers.Dense(2, activation='softmax' ))


In [22]:
simple_cnn_with_attention.summary()

In [23]:
simple_cnn_with_attention.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

In [24]:
history = simple_cnn_with_attention.fit(train_x, train_y, epochs=30)

In [25]:
print(f"Test accuracy: {simple_cnn_with_attention.evaluate(test_x,test_y)[1]}")

In [26]:
plt.plot(history.history["accuracy"])
# plt.plot(hist.history["val_accuracy"])
plt.title("model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["train", "validation"], loc="upper left")
plt.show()

In [ ]:
y_pred = history.predict()

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_confusion = confusion_matrix(y_test, y_pred)